# Drive Share Example Analysis

This is an outline for a combined documentation / example anaylsis of the driveshare.me project by Eric Hong.

## Downloading the Data

Anonymized path data can be obtained as a json file and high level data as a csv file from http://driveshare.me. 
Click `anonymized paths` or `high level data` in the bottom right.

Examples of how to use the data are shown below.

## Loading Data Directly into Julia

In [3]:
using DriveShare
anon_paths = get_dataset(:trace) # Anonymized paths now loaded.
typeof(anon_paths)

Array{Any,1}

Total distance calculated is less than what is shown on driveshare.me
because `get_dataset(:trace)` only returns trip objects that contain paths.
As explained in the [api-reference](https://developer.automatic.com/api-reference/): 
"In cases where no GPS signal was available during the trip, path may be returned as null."

![Ma Image](json.png)

Here we extract the total duration and distance of all trips.

In [4]:
total_duration_s = 0
total_distance_m = 0
for path_data in anon_paths
    total_duration_s += path_data["duration_s"]
    total_distance_m += path_data["distance_m"]
end
@printf("total duration (seconds): %15.3f\n", total_duration_s)
@printf("total distance (meters)): %15.3f\n", total_distance_m)

total duration (seconds):     2461558.700
total distance (meters)):    30729886.700


The High Level dataset is loaded the same way, but returns a DataFrame.

In [5]:
high_level = get_dataset(:highlevel) # DataFrame
names(high_level)

39-element Array{Symbol,1}:
 :trip_ID              
 :vehicle_ID           
 :start_time           
 :end_time             
 :distance_m           
 :duration_s           
 :fuel_cost            
 :fuel_volume          
 :average_kmpl         
 :average_from_epa_kmpl
 :score_events         
 :score_speeding       
 :hard_brakes          
 ⋮                     
 :battery_voltage      
 :active_dtcs          
 :start_city           
 :start_state          
 :start_country        
 :end_city             
 :end_state            
 :end_country          
 :start_lat            
 :start_long           
 :end_lat              
 :end_long             

In [6]:
using CrossfilterCharts
using DataFrames
df = DataFrame()
for sym in [:make, :model, :distance_m, :average_kmpl, :score_speeding, 
            :hard_brakes, :city_fraction, :highway_fraction]
    df[sym] = high_level[sym]
end

dc(df)

INFO: Recompiling stale cache file /home/tim/.julia/lib/v0.5/CrossfilterCharts.ji for module CrossfilterCharts.
  likely near /home/tim/.julia/v0.5/CrossfilterCharts/src/io.jl:250


CrossfilterCharts.DCOut(2702×8 DataFrames.DataFrame
│ Row  │ make     │ model     │ distance_m │ average_kmpl │ score_speeding │
├──────┼──────────┼───────────┼────────────┼──────────────┼────────────────┤
│ 1    │ "Subaru" │ "Outback" │ 13.6       │ 1.5          │ 50.0           │
│ 2    │ "Subaru" │ "Outback" │ 2002.2     │ 7.9          │ 50.0           │
│ 3    │ "Subaru" │ "Outback" │ 117.9      │ 3.2          │ 50.0           │
│ 4    │ "Subaru" │ "Outback" │ 224.7      │ 3.8          │ 50.0           │
│ 5    │ "Subaru" │ "Outback" │ 46572.0    │ 9.7          │ -15.25         │
│ 6    │ "Subaru" │ "Outback" │ 47357.4    │ 23.2         │ 50.0           │
│ 7    │ "Subaru" │ "Outback" │ 2293.3     │ 8.8          │ 50.0           │
│ 8    │ "Subaru" │ "Outback" │ 172.9      │ 5.7          │ 50.0           │
│ 9    │ "Subaru" │ "Outback" │ 18207.4    │ 9.8          │ 49.1           │
│ 10   │ "Subaru" │ "Outback" │ 23055.8    │ 11.9         │ 35.12          │
│ 11   │ "Subaru" │ "Outback" │ 14007.9    │ 15.5         │ 34.41          │
⋮
│ 2691 │ "Toyota" │ "Sienna"  │ 5387.3     │ 6.5          │ 50.0           │
│ 2692 │ "Toyota" │ "Sienna"  │ 1550.3     │ 5.9          │ 50.0           │
│ 2693 │ "Toyota" │ "Sienna"  │ 4847.5     │ 8.1          │ 50.0           │
│ 2694 │ "Toyota" │ "Sienna"  │ 5580.6     │ 9.5          │ 50.0           │
│ 2695 │ "Toyota" │ "Sienna"  │ 2451.4     │ 8.2          │ 50.0           │
│ 2696 │ "Toyota" │ "Sienna"  │ 3174.7     │ 8.5          │ 50.0           │
│ 2697 │ "Toyota" │ "Sienna"  │ 1785.6     │ 6.5          │ 50.0           │
│ 2698 │ "Toyota" │ "Sienna"  │ 1845.2     │ 9.3          │ 50.0           │
│ 2699 │ "Subaru" │ "Outback" │ 2500.8     │ 7.8          │ 50.0           │
│ 2700 │ "Subaru" │ "Outback" │ 2696.1     │ 8.5          │ 50.0           │
│ 2701 │ "Subaru" │ "Outback" │ 1672.2     │ 5.8          │ 50.0           │
│ 2702 │ "Subaru" │ "Outback" │ 17012.1    │ 9.6          │ 46.01          │

│ Row  │ hard_brakes │ city_fraction │ highway_fraction │
├──────┼─────────────┼───────────────┼──────────────────┤
│ 1    │ 0           │ 1.0           │ 0.0              │
│ 2    │ 0           │ 1.0           │ 0.0              │
│ 3    │ 0           │ 1.0           │ 0.0              │
│ 4    │ 0           │ 1.0           │ 0.0              │
│ 5    │ 1           │ 0.64          │ 0.36             │
│ 6    │ 0           │ 0.77          │ 0.23             │
│ 7    │ 0           │ 1.0           │ 0.0              │
│ 8    │ 0           │ 1.0           │ 0.0              │
│ 9    │ 1           │ 0.93          │ 0.07             │
│ 10   │ 1           │ 0.82          │ 0.18             │
│ 11   │ 0           │ 0.31          │ 0.69             │
⋮
│ 2691 │ 4           │ 1.0           │ 0.0              │
│ 2692 │ 0           │ 1.0           │ 0.0              │
│ 2693 │ 0           │ 1.0           │ 0.0              │
│ 2694 │ 0           │ 1.0           │ 0.0              │
│ 2695 │ 0           │ 1.0           │ 0.0              │
│ 2696 │ 0           │ 1.0           │ 0.0              │
│ 2697 │ 0           │ 1.0           │ 0.0              │
│ 2698 │ 0           │ 1.0           │ 0.0              │
│ 2699 │ 0           │ 1.0           │ 0.0              │
│ 2700 │ 1           │ 1.0           │ 0.0              │
│ 2701 │ 0           │ 1.0           │ 0.0              │
│ 2702 │ 0           │ 0.95          │ 0.05             │,CrossfilterCharts.Dimension[CrossfilterCharts.Dimension(:make,"function(d){return d.make; }",NaN),CrossfilterCharts.Dimension(:model,"function(d){return d.model; }",NaN),CrossfilterCharts.Dimension(:distance_m,"function(d){return Math.round(d.distance_m / 50000.000000)*50000.000000; }",50000.0),CrossfilterCharts.Dimension(:average_kmpl,"function(d){return Math.round(d.average_kmpl / 1.000000)*1.000000; }",1.0),CrossfilterCharts.Dimension(:score_speeding,"function(d){return Math.round(d.score_speeding / 100.000000)*100.000000; }",100.0),CrossfilterCharts.Dimensio

## Basic Data Analysis

In [7]:
@printf("%-50s %10d\n", "number of trips:", size(high_level, 1))
@printf("%-50s %10d\n", "number of unique vehicles:", length(unique(high_level[:vehicle_ID])))
@printf("%-50s %14.3f\n", "total duration [sec]", sum(high_level[:duration_s]))
@printf("%-50s %14.3f\n", "total distance [m]", sum(high_level[:distance_m]))
@printf("%-50s %10d\n", "number of unique vehicle models:", length(unique(high_level[:model])))

number of trips:                                         2702
number of unique vehicles:                                  5
total duration [sec]                                  2470401.700
total distance [m]                                   30800351.900
number of unique vehicle models:                            4


## Number of Trips per Vehicle

In [20]:
include("plot_helper.jl")
plot_number_of_trips_per_vehicle(high_level)

## Trip Distance per Vehicle


In [9]:
plot_histogram_per_vehcile(high_level, :distance_m)

## Fuel Cost Histogram per Vehicle 

In [10]:
plot_histogram_per_vehcile(high_level, :fuel_cost)

## Hard Accelerations per Vehicle

In [11]:
plot_histogram_per_vehcile(high_level, :hard_accels)

## Hard Brakes per Vehicle

In [12]:
plot_histogram_per_vehcile(high_level, :hard_brakes)

## Speeding Score per Vehicle

"driving score for speeding"

In [13]:
plot_histogram_per_vehcile(high_level, :score_speeding)

## Vehicle Comparison

In [14]:
targets = [:distance_m, :duration_s, :average_kmpl, :score_speeding,  :hard_brakes, :city_fraction, :highway_fraction,
    :duration_over_70_s, :duration_over_75_s, :duration_over_80_s, :idling_time_s]
targetA = targets[5]
targetB = targets[8]
scatter_targets(high_level, targetA, targetB)

## Path Data

Here we create a histogram of separation distances between GPS points over all trips.

In [15]:
separation_distances = []
for path_data in anon_paths
    for i = 1:length(path_data["utm"])-1
        d = norm(path_data["utm"][i] - path_data["utm"][i+1])
        push!(separation_distances, d)
    end
end
histogram(filter(x->x < 150, separation_distances), nbins=70, xlabel="separation distance [m]", ylabel="count", size=(900,400), leg=false)

Here we create a histogram of (separation distance) / (mean trip velocity), an approximation of the time passed between each point, between GPS points over all trips.

In [16]:
counter = 0
separation_times = []
for path_data in anon_paths
    mean_trip_velocity = path_data["distance_m"] / path_data["duration_s"] # meters/second
    for i = 1:length(path_data["utm"])-1
        counter += 1
        t = separation_distances[counter] / mean_trip_velocity # seconds
        push!(separation_times, t)
    end
end
histogram(filter(x->x < 15, separation_times), nbins=70, xlabel="approx. time between points [s]", ylabel="count", size=(900,400), leg=false)

In [19]:
plot_drive(3)